In [90]:
from src import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Snippet to fit a model on a graph generated on the fly.

First, we generate a graph and the accompanying choice sets. In this case, we set $p=0.25$.

In [ ]:
# generate a growing graph
(G, el) = network_generation.generate_mixed_model('test', n_max=20000, r=1.00, p=0.25, grow=True, m=4, vvv=1)
# compute choice sets
D = network_stats.choice_data('test', el, vvv=1)

[test] generated a growing graph with 5004 nodes and 20001 edges (r=1.00, p=0.25)


Then, we try fitting a few different models.

## 1) Log Logit

In [ ]:
# fit single log model
m1 = logit_individual.LogLogitModel('test', D=D, vvv=2)
m1.fit()

In [ ]:
plt.figure()
xs = np.arange(50)
plt.fill_between(xs, xs * (m1.u - 1.96*m1.se), xs * (m1.u + 1.96*m1.se), color='lightgrey')
plt.plot(xs, xs * m1.u, '-b')
plt.xlabel(r"$k$")
plt.ylabel(r"$u_k$")

## 2) Degree Logit

In [ ]:
# fit degree logit model
m2 = logit_individual.DegreeLogitModel('test', D=D, vvv=2)
m2.fit()

In [ ]:
plt.figure()
xs = np.arange(51)
plt.fill_between(xs, np.exp(m2.u - 1.96*m2.se), np.exp(m2.u + 1.96*m2.se), color='lightgrey')
plt.plot(xs, np.exp(m2.u), '-b')
plt.xlabel(r"$k$")
plt.ylabel(r"$u_k$")
axes = plt.gca()
axes.set_ylim([0, 10])

## 3) Mixed Logit

In [ ]:
# fit mixed logit model
m3 = logit.MixedLogitModel('test', grouped=False, D=D, vvv=vvv)
m3.add_log_model()  # 'unclamped'
#m.add_log_model(bounds=((1, 1)),)  # 'clamped at alpha=1'
m3.add_poly_model(k=1, bounds=((1, 1),))  # uniform
T = m3.fit(etol=0.05, n_rounds=50, return_stats=True)

In [ ]:
plt.figure(figsize=(9,3))

plt.subplot(131)
T.tot_ll.plot()
plt.title('Total log likelihood')

plt.subplot(132)
T.p1.plot()
plt.title(r'Class probability $\pi$')

plt.subplot(133)
T.u1.plot()
plt.title(r'Estimate of $\alpha$')

plt.tight_layout()